# Word2Vec笔记
——图表示学习的基础，其思想被广泛应用于捕捉图节点的embedding中。

>author: xumayi@m.scnu.edu.cn

**Note: Latex公式需在本地查看**

## Reference
#### Source papers
[1] [Mikolov T, Chen K, Corrado G, et al. Efficient estimation of word representations in vector space[J]. arXiv preprint arXiv:1301.3781, 2013.](https://arxiv.org/pdf/1301.3781.pdf)
Main contributon:Proposing the Word2Vec.

[2] [Le Q, Mikolov T. Distributed representations of sentences and documents[C]//International conference on machine learning. PMLR, 2014: 1188-1196.](http://proceedings.mlr.press/v32/le14.pdf)
Main contributon:Details of the two tricks in Word2Vec: hierarchical softmax and negative sampling.

#### Explasion papers (Showing some details of Word2Vec)
[3] [Rong X. word2vec parameter learning explained[J]. arXiv preprint arXiv:1411.2738, 2014.](https://arxiv.org/pdf/1411.2738.pdf)
#### Blogs
1、[秒懂词向量Word2vec的本质](https://zhuanlan.zhihu.com/p/26306795/)

## 1、Word2Vec的简单情形
目标：用当前词x预测下一个词语y

<img src="https://pic4.zhimg.com/80/v2-a1a73c063b32036429fbd8f1ef59034b_1440w.jpg" width = "400">

输入：当前词的One-hot表示 $w_k$

输出目标：预测词的One-hot表示 $w_j$

训练之后得到:

输入矩阵 $W_{V \times N} \in \mathbb{R}^{V \times N}$, 输出矩阵$W^{\prime}_{V \times N} \in \mathbb{R}^{V \times N}$

V是词典的大小，N是隐藏层的维度，通常$N \ll V$

假定输入：$x = [x_1,x_2,...,x_k,...,x_V]^T \in \mathbb{R}^{V \times 1} $ 其中 $x_k=1$ 且 $x_{k^{\prime}}=0$ for $k^{\prime} \neq k$

隐藏层: $h=[h_1,h_2...,h_N]^T \in \mathbb{R}^{N \times 1}$

$h=W_{V \times N}^Tx =v_{k}^T \in \mathbb{R}^{N \times 1} \tag{1}$

这里$v_k$是$W_{V \times N}$中的第k行，所以该矩阵乘法实际上就是copy了$W_{V \times N}$的第$k$行。

同时，由隐藏层至输出层，输出层是需要计算每一个单词的概率，概率最大的单词即为预测出的结果。

假定$u_j$是第$j$个unit的值，$U=[u_1,u_2...,u_V]^T$

$u_j = v_j^{\prime}h \in \mathbb{R}^{1 \times 1} \tag{2}$ ,其中$v_j^{\prime}$为$W_{N \times V}$的第$j$行

经过$softmax$（此处暂时不考虑hieracerical softmax和负采样这两种优化策略）
: $y_j = \frac{exp(u_j)}{\sum_{j^{\prime}=1}^V exp(u_{j^{\prime}})} \tag{3}$

其中$y_j$是输出层上第$j$个unit的值，也就是词典中第$j$个单词为预测单词的概率

将公式（1）代入公式（2）有：$u_j=v_j^{\prime}v_k^T \tag{4}$

将公式（4）代入（3）有：$y_j = \frac{exp(v_j^{\prime}v_k^T)}{\sum_{j^{\prime}=1}^V exp(v_{j^{\prime}}^{\prime}v_k^T)}$

优化目标：$max(p(w_j|w_o)) = max(y_j) = max(log(y_j)) \tag{5}$

将公式（3）代入公式（5）有：$max(p(w_j|w_o)) = max (log (\frac{exp(u_j)}{\sum_{j^{\prime}=1}^V exp(u_{j^{\prime}})})) = max(u_j - log \sum_{j^{\prime}=1}^V exp(u_{j^{\prime}}))$

$u_j - log \sum_{j^{\prime}=1}^V exp(u_{j^{\prime}}) := -E$

$max(-E)=min(E)$,优化目标即变为$min(log \sum_{j^{\prime}=1}^V exp(u_{j^{\prime}}) - u_j)$

即loss函数为$log \displaystyle\sum_{j^{\prime}=1}^V exp(u_{j^{\prime}}) - u_j$

## 2、CBOW

<img src='https://pic3.zhimg.com/v2-d1ca2547dfb91bf6a26c60782a26aa02_r.jpg' width='400'>

同时用多个上下文单词预测一个单词，输入矩阵共享权重

由输入层到隐藏层：$h=\frac{1}{C}W_{V \times N}^{T} (x_1+x_2+x_3+...+x_C)=\frac{1}{C}(v_{w1}+v_{w2}+...+v_{wC})^T \tag{6}$

由隐藏层到输出层和简单情形一样，即$u_j = v^\prime_jh$，将公式（6）代入该公式中有

$u_j=\frac{1}{C}v^\prime_j(v_{w1}+v_{w2}+...+v_{wC})^T$

损失函数：$log \displaystyle\sum_{j^{\prime}=1}^V exp(u_{j^{\prime}}) - u_j$

## 3、Skip-gram

<img src='https://pic1.zhimg.com/v2-ca81e19caa378cee6d4ba6d867f4fc7c_r.jpg' width='400'>

用一个单词预测上下文单词，输出矩阵共享权重

由输入层到隐藏层与简单情形一致：$h = W_{V \times N}^Tx=v^k_T \in \mathbb{R}^{N \times 1}$

$u_{c,j^*_c}=v^{\prime^T}_{w_{j^*}}h,(c=1,2...,C)$,

输出层中，每一个输出：$y_{c,j^*_c}= \frac{exp(u_{c,j^*_c})}{\sum_{j^{\prime}=1}^{v}exp(u_{j^\prime})}$,其中$u_{c,j^*}$是第$c$个$(c=1,2,...,C)$要预测的上下文单词的one-hot向量中为1的位置对应的unit的值。（$j^*_c$在每个上下文单词中不同）

损失函数$E=-logp(w_{O,1},w_{O,2} ,...,w_{O,C}|w_I)=-log \prod^C_{c=1}\frac{exp(u_{c,j^{*}_c})}{\sum^V_{j^{\prime}=1}exp(u_j^{\prime})}=-\sum^C_{c=1}u_{j^*_c}+Clog\sum^V_{j^\prime=1}exp(u_{j^\prime})$


## 4、Hierarchical Softmax

在CBOW和Skip-gram中，有隐藏层到输出层，最有要经过一层Softmax的计算，显然这种开销是
极其昂贵的，如公式（3）所示，$y_j = \frac{exp(u_j)}{\sum_{j^{\prime}=1}^V exp(u_{j^{\prime}})}$
，其中$V$为vocalubary的大小，在计算过程中，由于实际的$V$很大，所以$\sum_{j^{\prime}=1}^V exp(u_{j^{\prime}})$
的计算量极大，这给直接在输出层中采用Softmax计算造成了十分大的开销，因此本论文采用了Hierarchical Softmax的方法来减少计算开销。

* Huffman Tree（哈夫曼树）：具有最小带权路径长度的二叉树称为哈夫曼树（也称最优二叉树）。

假定我们有(a，b，c，d，e，f)共6个节点，节点的权值分布是(16，4，8，6，20，3)，那么根据这几个节点的权值所构建的哈夫曼树如下图所示。

<img src ='https://img-blog.csdnimg.cn/20200729110222289.png' width='500'>

同样，在文本上，我们可以根据每个词在语料中出现的次数来构建哈夫曼树。例如文本"喜欢观看巴西足球世界杯"，其中各个单词的频率为{"喜欢"：8，"观看"：6，"巴西"：5，"足球"：3，"世界杯"：1}，根据其构建的哈夫曼树如下图隐藏层到输出层中间所示。
每个叶子节点代表语料库中的一个词，于是每个词语都可以被01唯一的编码，并且其编码序列对应一个事件序列。

<img src='https://img-blog.csdnimg.cn/20200729110548332.png' width='500'>

我们可以计算条件概率 $p(w \mid \operatorname{Context}(w))$

在开始计算之前, 还是得引入一些符号:
1. $p^{w}$ :从根结点出发到达 $w$ 对应叶子结点的路径
2. $l^{w}$ : 路径中包含结点的个数
3. $p_{1}^{w}, p_{2}^{w}, \cdots, p_{l^{w}}^{w}$ :路径 $p^{w}$ 中的各个节点
4. $d_{2}^{w}, d_{3}^{w}, \cdots, d_{l w}^{w} \in\{0,1\}$ :词 $\mathrm{w}$ 的编码, $\quad d_{j}^{w}$ 表示路径 $p^{w}$ 第 $\mathrm{j}$ 个节点对应的编码（根节点无编码，左1右0)
5. $\theta_{1}^{w}, \theta_{2}^{w}, \cdots, \theta_{l^{w}-1}^{w} \in \mathbb{R}^{m}$ :路径 $p^{w}$ 中非叶节点对应的参数向量

于是可以给出 $w$ 的条件概率:
$p(w \mid$ Context $(w))=\prod_{j=2}^{l^{w}} p\left(d_{j}^{w} \mid \mathbf{x}_{w}, \theta_{j-1}^{w}\right)$

对于 $w=$ '足球', 其长度 $l^{w}=5, p_{1}^{w}, p_{2}^{w}, p_{3}^{w}, p_{4}^{w}, p_{5}^{w}$ 为路径 $p^{w}$ 上的
5 个结点,其中 $p_{1}^{w}$ 对应根结点。 $d_{2}^{w}, d_{3}^{w}, d_{4}^{w}, d_{5}^{w}$ 分别为 $1,0,0,1$ 即 "足球"的 Huffman编码为 1001 ;
此外, $\theta_{1}^{w}, \theta_{2}^{w}, \theta_{3}^{w}, \theta_{4}^{w}$ 分别表示路径 $\mathrm{p}^{w}$ 上 4 个非叶子结点对应的向量。
在 word2vec 中, 1 表示负类（左侧分支）, 0 表示正类（右侧分支）。通过公式Sigmoid 函数对其进行分类得:

分为正类的概率: $\sigma\left(x_{w}^{T} \theta\right)=\frac{1}{1+e^{-x_{\mathrm{w}}^{\tau} \theta}}$; 则分为负类的概率: $1-\sigma\left(x_{w}^{T} \theta\right)$

其中, 每一项是一个Logistic Regression:

$p\left(d_{j}^{w} \mid x_{w}, \theta_{j-1}^{w}\right)=\left\{\begin{array}{ll}\sigma\left(x_{w}^{\top} \theta_{j-1}^{w}\right), & d_{j}^{w}=0 \\ 1-\sigma\left(x_{w}^{\top} \theta_{j-1}^{w}\right), & d_{j}^{w}=1\end{array}\right.$
考虑到d只有0和1两种取值, 我们可以用指数形式方便地将其写到一起;

$p\left(d_{j}^{w} \mid \mathbf{x}_{w}, \theta_{j-1}^{w}\right)=\left[\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]^{1-d_{j}^{w}} \cdot\left[1-\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]^{d^{w}}$

我们的目标函数取对数似然:
$\mathcal{L}=\sum_{w \in \mathcal{C}} \log p(w \mid \text { Context }(w))$
将 $p(w \mid \operatorname{Context}(w))$ 代入上式, 有:

\begin{array}{l}
\mathcal{L}=\sum_{w \in \mathcal{C}} \log \prod_{j=2}^{l^{w}}\left\{\left[\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]^{1-d_{j}^{w}} \cdot\left[1-\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]^{d_{j}^{w}}\right\} \\
=\sum_{w \in \mathcal{C}} \sum_{j=2}^{l^{w}}\left\{\left(1-d_{j}^{w}\right) \cdot \log \left[\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]+d_{j}^{w} \cdot \log \left[1-\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]\right\}
\end{array}

这也很直白, 连乘的对数换成求和。不过还是有点长, 我们把每一项简记为:
\begin{equation}
\mathcal{L}(w, j)=\left(1-d_{j}^{w}\right) \cdot \log \left[\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]+d_{j}^{w} \cdot \log \left[1-\sigma\left(\mathbf{x}_{w}^{\top} \theta_{j-1}^{w}\right)\right]
\end{equation}
